<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/traducci%C3%B3n_autom%C3%A1tica/traducci%C3%B3n_ingl%C3%A9s_a_espa%C3%B1ol_con_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traducción Automática de Inglés a Español

**Objetivo:**

Entrenar un Transformador *seq2seq* (secuencia a secuencia) para resolver una tarea de traducción automática de inglés a español.

Este problema es conocido por sus siglas en inglés como NMT (*Neural Machine Translation*)

## Introducción

En este *notebook* se diseña un modelo de tipo Transformador de secuencia a secuencia utilizando la API funcional de Tensorflow, que será entrenado en una tarea de traducción automática de inglés a español.

Se implementarán los siguientes puntos:

- Preparar los datos para entrenar un modelo *seq2seq* donde la secuencia de entrada es una oración en inglés y la secuencia de salida será la oración traducida a español
- Vectorizar el texto usando la clase `TextVectorization` de Keras
- Implementar una clase `PositionalEmbedding`
- Implementar una clase `TransformerEncoder`
- Implementar una clase `TransformerDecoder`
- Entrenar el modelo para resolver una tarea de traducción automática
- Utilizar el modelo para generar traducciones a oraciones que no fueron utilizadas durante el entrenamiento

In [1]:
# Características de Hardware
!nvidia-smi

Mon Jan  9 03:37:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Librerías

In [2]:
import re
import random
import string
import pathlib
import numpy as np
from typing import Optional, Dict, Any

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.callbacks import ModelCheckpoint

## Conjunto de datos

En el siguiente enlace se pueden encontrar diversos conjuntos de datos de oraciones en inglés traducidas a varios idiomas: https://www.manythings.org/anki/.

En este caso se utilizará el conjunto de datos correspondiente a inglés - español.

In [3]:
text_file = tf.keras.utils.get_file(fname="spa-eng.zip",
                                    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
                                    extract=True)

text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

## Preprocesamiento de datos

Cada línea del conjunto de datos contiene una oración en inglés y su correspondiente oración traducida a español.
La oración en inglés es la secuencia de origen y la oración en español es la secuencia de destino.
Anteponemos el *token* `"[start]"` y agregamos el token `"[end]"` a la oración en español.

In [4]:
# Abrir el archivo de texto especificado
with open(text_file) as f:
    # Leer las líneas del archivo y dividirlas en una lista
    lines = f.read().split("\n")[:-1]

# Inicializar una lista para almacenar los pares de texto
text_pairs = []

# Para cada línea en el archivo
for line in lines:
    # Dividir en los idiomas inglés y español
    eng, spa = line.split("\t")
    # Agregar marcadores de inicio y fin al texto en español
    spa = "[start] " + spa + " [end]"
    # Agregar los pares de texto a la lista
    text_pairs.append((eng, spa))

Así es como se ven los pares de oraciones (inglés, español) generados por la celda previa para 5 registros:

In [5]:
for _ in range(5):
    print(random.choice(text_pairs), '\n')

('I will be in high school next April.', '[start] Estaré en secundaria el próximo abril. [end]') 

('Tom has bad table manners.', '[start] Tom tiene malos modales en la mesa. [end]') 

('How is everything?', '[start] ¿Cómo va todo? [end]') 

('I was at home almost all day yesterday.', '[start] Ayer estuve casi todo el día en casa. [end]') 

("That was the tiniest cockroach I've ever seen in my life.", '[start] Aquella fue la cucaracha más pequeña que he visto en mi vida. [end]') 



Ahora, se particionan los pares de oraciones en los conjuntos de entrenamiento, validación, y prueba.

In [6]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"[INFO] {len(text_pairs)} Pares totales (inglés, español)")
print(f"[INFO] {len(train_pairs)} Pares en el conjunto de entrenamiento")
print(f"[INFO] {len(val_pairs)} Pares en el conjunto de validación")
print(f"[INFO] {len(test_pairs)} Pares en el conjunto de prueba")

[INFO] 118964 Pares totales (inglés, español)
[INFO] 83276 Pares en el conjunto de entrenamiento
[INFO] 17844 Pares en el conjunto de validación
[INFO] 17844 Pares en el conjunto de prueba


## Vectorización de texto

Se utilizan dos instancias de la clase `TextVectorization` para vectorizar los datos de texto (una para inglés y otra para español), es decir, se convierten las cadenas originales en texto a secuencias de números enteros donde cada entero representa el índice de una palabra en un vocabulario.

La clase en inglés utilizará la estandarización de cadenas predeterminada (eliminar los caracteres de puntuación) y el enfoque de separación (separar por espacios en blanco), mientras que en la clase en español se utilizará una estandarización personalizada, donde se agregará el caracter `"¿"` al conjunto de caracteres de puntuación **a eliminar**.

Importante: La limpieza de datos propuesta es sólo con fines didácticos, pues en caso de tener como objetivo el despliegue a producción de un modelo de traducción automática, no se recomendaría eliminar los caracteres de puntuación en independencia del idioma que se esté modelando. En su lugar, convertir cada carácter de puntuación a su propio *token*.

In [7]:
def custom_standardization(input_string):
    """Realiza la estandarización de una cadena de entrada.
    Primero, convierte la cadena a minúsculas. Luego, elimina todos los caracteres especificados en la variable
    global `strip_chars` utilizando expresiones regulares.

    Argumentos:
        input_string: Una cadena de tensores, la cadena de entrada.

    Devuelve:
        Una cadena de tensores, la cadena estandarizada.
    """
    strip_chars = string.punctuation + "¿"
    strip_chars = strip_chars.replace("[", "")
    strip_chars = strip_chars.replace("]", "")

    # Convertir la cadena a minúsculas
    lowercase = tf.strings.lower(input_string)
    # Eliminar los caracteres especificados en `strip_chars` utilizando expresiones regulares
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

In [8]:
batch_size = 64
vocab_size = 15000
sequence_length = 20

eng_vectorization = TextVectorization(max_tokens=vocab_size,
                                      output_mode="int",
                                      output_sequence_length=sequence_length)

spa_vectorization = TextVectorization(max_tokens=vocab_size,
                                      output_mode="int",
                                      output_sequence_length=sequence_length + 1,
                                      standardize=custom_standardization)

train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

Ahora, se formatearán las particiones de datos.

En cada paso de entrenamiento, el modelo buscará predecir las palabras objetivo (*targets*) N+1 y consecuentes usando la oración de origen (*inputs*) y las palabras objetivo 0 a N.

El conjunto de datos de entrenamiento generará una tupla `(inputs, targets)`, donde:

- `inputs` es un diccionario con las llaves `encoder_inputs` y `decoder_inputs`. `encoder_inputs` es la oración de origen vectorizada y `decoder_inputs` es la oración de destino "hasta ahora", es decir, las palabras 0 a N utilizadas para predecir la palabra N+1 (y consecuentes) en la oración de destino.
- `target` es la oración de destino desplazada por un paso: proporciona las siguientes palabras en la oración de destino, o dicho de otra forma, lo que el modelo intentará predecir.

In [9]:
def format_dataset(eng, spa):
    """Formatear el conjunto de datos para entrenamiento.
    Argumentos:
    eng: Un tensor, el tensor de entrada del encoder.
    spa: Un tensor, el tensor de entrada del decoder.

    Devuelve:
        Un par de tensores, el tensor de entrada para el modelo y el tensor de salida para el modelo.
    """
    # Vectorizar la entrada del encoder
    eng = eng_vectorization(eng)
    # Vectorizar la entrada del decoder
    spa = spa_vectorization(spa)
    # Devolver el par de tensores formateados
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])

In [10]:
def make_dataset(pairs):
    """Crear un dataset a partir de pares de texto en inglés y español.
    Argumentos:
    pairs: Una lista de tuplas, cada tupla contiene un texto en inglés y otro en español.

    Devuelve:
        Un dataset, el dataset creado.
    """
    # Desempaquetar la lista de pares en dos listas, una con textos en inglés y otra con textos en español
    eng_texts, spa_texts = zip(*pairs)
    # Convertir las listas a listas de Python
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    # Crear un dataset a partir de las listas de textos
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    # Agrupar los elementos del dataset en batches
    dataset = dataset.batch(batch_size)
    # Aplicar la función de formateo al dataset
    dataset = dataset.map(format_dataset)
    # Mezclar los elementos del dataset, prefetchear 16 elementos y cachear los resultados
    return dataset.shuffle(2048).prefetch(16).cache()

In [11]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Validación de dimensiones de la secuencia. (Se toman lotes de 64 pares y todas las secuencias tienen 20 pasos):

In [12]:
for inputs, targets in train_ds.take(1):
    print(f"[INFO] inputs['encoder_inputs'].shape: {inputs['encoder_inputs'].shape}")
    print(f"[INFO] inputs['decoder_inputs'].shape: {inputs['decoder_inputs'].shape}")
    print(f"[INFO] targets.shape: {targets.shape}")

[INFO] inputs['encoder_inputs'].shape: (64, 20)
[INFO] inputs['decoder_inputs'].shape: (64, 20)
[INFO] targets.shape: (64, 20)


## Construcción del modelo

Nuestro Transformador de secuencia a secuencia consiste en un `TransformerEncoder` y un `TransformerDecoder` encadenados juntos. Para que el modelo sea consciente del orden de las palabras, también usamos una capa `PositionalEmbedding`.

La secuencia fuente se pasará al `TransformerEncoder`, que producirá una nueva representación de la misma. Esta nueva representación se pasará al `TransformerDecoder`, junto con la secuencia de destino hasta el momento (palabras de destino 0 a N). El 'TransformerDecoder' intentará predecir las siguientes palabras en la secuencia de destino (N+1 y más allá).

Un detalle clave que hace esto posible es el enmascaramiento causal (consulte el método `get_causal_attention_mask()` en `TransformerDecoder`). El `TransformerDecoder` ve las secuencias completas a la vez y, por lo tanto, debemos asegurarnos de que solo use información de los tokens de destino 0 a N al predecir el token N+1 (de lo contrario, podría usar información del futuro, lo que daría como resultado un modelo que no se puede utilizar en el momento de la inferencia).

In [13]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length: int, vocab_size: int, embed_dim: int, **kwargs):
        """Inicializar la clase PositionalEmbedding.
        Argumentos:
            sequence_length: Entero, la longitud de la secuencia.
            vocab_size: Entero, el tamaño del vocabulario.
            embed_dim: Entero, el tamaño de las incrustaciones.
            **kwargs: Otros argumentos clave.
        """
        # Inicializar la clase
        super().__init__(**kwargs)
        # Inicializar las incrustaciones para los tokens y las posiciones
        self.token_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        # Guardar los atributos de la clase
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs: tf.Tensor) -> tf.Tensor:
        """Realizar el pase hacia adelante de la capa PositionalEmbedding.

        Argumentos:
            inputs: Un tensor, el tensor de entrada de la capa.

        Devuelve:
            Un tensor, el tensor de salida de la capa.
        """
        # Obtener la longitud de los inputs
        length = tf.shape(inputs)[-1]
        # Generar un tensor con las posiciones
        positions = tf.range(start=0, limit=length, delta=1)
        # Incorporar incrustaciones a los tokens
        embedded_tokens = self.token_embeddings(inputs)
        # Incorporar incrustaciones a las posiciones
        embedded_positions = self.position_embeddings(positions)
        # Sumar las incrustaciones de los tokens y posiciones
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs: tf.Tensor, mask: Optional[tf.Tensor] = None) -> tf.Tensor:
        """Calcular la máscara para la capa PositionalEmbedding.

        Argumentos:
            inputs: Un tensor, el tensor de entrada de la capa.
            mask: Un tensor, la máscara a aplicar a la capa.

        Devuelve:
            Un tensor, la máscara para la capa.
        """
        # Devolver una máscara que indique dónde hay tokens presentes
        return tf.math.not_equal(inputs, 0)

    def get_config(self) -> Dict[str, Any]:
        """Devuelve la configuración de la capa PositionalEmbedding.

        Devuelve:
            Un diccionario, la configuración de la capa.
        """
        # Obtener la configuración de la clase padre
        config = super().get_config()
        # Actualizar la configuración con los atributos de esta clase
        config.update({
        "sequence_length": self.sequence_length,
        "vocab_size": self.vocab_size,
        "embed_dim": self.embed_dim,
        })
        # Devolver la configuración actualizada
        return config

In [14]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim: int, dense_dim: int, num_heads: int, **kwargs):
        """Inicializar la capa TransformerEncoder.

        Argumentos:
            embed_dim: Entero, el tamaño de las incrustaciones.
            dense_dim: Entero, el tamaño de la capa densa.
            num_heads: Entero, el número de cabezas de atención.
        """
        # Inicializar la clase
        super().__init__(**kwargs)
        # Asignar el tamaño de las incrustaciones a una variable de instancia
        self.embed_dim = embed_dim
        # Asignar el tamaño de la capa densa a una variable de instancia
        self.dense_dim = dense_dim
        # Asignar el número de cabezas de atención a una variable de instancia
        self.num_heads = num_heads
        # Crear una capa de atención multi-cabeza
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # Crear una secuencia de capas densas
        self.dense_proj = tf.keras.Sequential([layers.Dense(dense_dim, activation="relu"),
                                               layers.Dense(embed_dim),])
        # Crear una capa de normalización de capa
        self.layernorm_1 = layers.LayerNormalization()
        # Crear otra capa de normalización de capa
        self.layernorm_2 = layers.LayerNormalization()
        # Establecer la propiedad de soporte de máscara en verdadero
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, mask: Optional[tf.Tensor] = None) -> tf.Tensor:
        """Realizar el pase hacia adelante de la capa TransformerEncoder.

        Argumentos:
            inputs: Un tensor, el tensor de entrada de la capa.
            mask: Un tensor, la máscara a aplicar a la atención.

        Devuelve:
            Un tensor, el tensor de salida de la capa.
        """
        # Calcular la máscara de atención a partir de la máscara dada
        attention_mask = self.attention.compute_mask(mask)
        # Calcular la salida de atención a partir de las entradas y la máscara
        attention_output = self.attention(query=inputs,
                                          value=inputs,
                                          key=inputs,
                                          attention_mask=attention_mask)
        # Aplicar la primera capa de normalización de capa y sumarla a la salida de atención
        proj_input = self.layernorm_1(inputs + attention_output)
        # Calcular la salida de la secuencia de capas densas a partir de la entrada proyectada
        proj_output = self.dense_proj(proj_input)
        # Aplicar la segunda capa de normalización de capa y sumarla a la salida proyectada
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self) -> Dict[str, Any]:
        """Devuelve la configuración de la capa TransformerEncoder.

        Devuelve:
            Un diccionario, la configuración de la capa.
        """
        # Obtener la configuración de la clase base
        config = super().get_config()
        # Actualizar la configuración con las variables de instancia de la capa TransformerEncoder
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        # Devolver la configuración actualizada
        return config

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim: int, latent_dim: int, num_heads: int, **kwargs):
        """Inicializar la capa TransformerDecoder.

        Argumentos:
            embed_dim: Entero, el tamaño de las incrustaciones.
            latent_dim: Entero, el tamaño del espacio latente.
            num_heads: Entero, el número de cabezas de atención.
        """
        # Inicializar la clase
        super().__init__(**kwargs)
        # Guardar la dimensión de las incrustaciones, la dimensión latente y el número de cabezas de atención
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        # Crear dos capas de atención multi-cabeza
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # Crear una secuencia de capas densas que proyectará el tensor de salida
        self.dense_proj = tf.keras.Sequential([layers.Dense(latent_dim, activation="relu"),
                                               layers.Dense(embed_dim)])
        # Crear tres capas de normalización
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # Indicar que esta capa soporta el uso de máscaras
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, encoder_outputs: tf.Tensor, mask: Optional[tf.Tensor] = None) -> tf.Tensor:
        """Realiza el pase hacia adelante del decodificador Transformer.

        Argumentos:
            inputs: Un tensor, el tensor de entrada del decodificador.
            encoder_outputs: Un tensor, el tensor de salida del codificador.
            mask: Un tensor, la máscara a aplicar a la capa.

        Devuelve:
            Un tensor, el tensor de salida del decodificador.
        """
        # Obtener la máscara de atención causal
        causal_mask = self.get_causal_attention_mask(inputs)

        # Aplicar la máscara de padding si se proporciona
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        # Realizar el pase hacia adelante de la primera atención múltiple
        attention_output_1 = self.attention_1(query=inputs,
                                              value=inputs,
                                              key=inputs,
                                              attention_mask=causal_mask)
        out_1 = self.layernorm_1(inputs + attention_output_1)

        # Realizar el pase hacia adelante de la segunda atención múltiple
        attention_output_2 = self.attention_2(query=out_1,
                                              value=encoder_outputs,
                                              key=encoder_outputs,
                                              attention_mask=padding_mask)
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        # Realizar el pase hacia adelante de la capa densa
        proj_output = self.dense_proj(out_2)

        # Devolver el pase hacia adelante final
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs: tf.Tensor) -> tf.Tensor:
        """Genera una máscara causal para la atención en el decodificador del Transformer.
           La máscara se utilizará para restringir la atención del decodificador a solo
           los tokens anteriores al token actual durante el proceso de pase hacia adelante.

        Argumentos:
            inputs: Un tensor, el tensor de entrada del decodificador.

        Devuelve:
            Un tensor, la máscara causal para la atención en el decodificador.
        """
        # Obtener las dimensiones del tensor de entrada
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        # Generar una máscara con una diagonal de 1's y 0's
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        # Duplicar la máscara para cada elemento del batch
        mult = tf.concat([tf.expand_dims(batch_size, -1),
                          tf.constant([1, 1], dtype=tf.int32)],
                        axis=0)
        return tf.tile(mask, mult)

    def get_config(self) -> Dict[str, Any]:
        """Devuelve la configuración de la capa TransformerDecoder.

        Devuelve:
            Un diccionario, la configuración de la capa.
        """
        # Obtener la configuración de la clase base
        config = super().get_config()
        # Actualizar la configuración con las variables de instancia de la capa TransformerDecoder
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        # Devolver la configuración actualizada
        return config

## Ensamblaje del modelo

In [16]:
num_heads = 8
embed_dim = 256
latent_dim = 2048

# Crear un tensor de entrada para el encoder con una forma dinámica (None) y tipo de datos "int64"
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

# Aplicar la capa PositionalEmbedding al tensor de entrada del encoder
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Aplicar la capa TransformerEncoder al tensor resultante
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

# Crear un modelo para el encoder utilizando el tensor de entrada y el tensor de salida
encoder = tf.keras.Model(encoder_inputs, encoder_outputs)

# Crear un tensor de entrada para el decoder con una forma dinámica (None) y tipo de datos "int64"
decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")

# Crear un tensor de entrada para el estado del decoder con una forma dinámica (None) y tamaño de incrustación
encoded_seq_inputs = tf.keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")

# Aplicar la capa PositionalEmbedding al tensor de entrada del decoder
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Aplicar la capa TransformerDecoder al tensor resultante y al tensor de estado del decoder
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)

# Aplicar una capa de Dropout al tensor resultante
x = layers.Dropout(0.5)(x)

# Aplicar una capa Dense con función de activación "softmax" al tensor resultante
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Crear un modelo para el decoder utilizando los tensores de entrada y el tensor de salida
decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

# Obtener el tensor de salida del decoder utilizando los tensores de entrada del decoder y el encoder
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

# Crear un modelo con las entradas del codificador y del decodificador, y las salidas del decodificador
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")

## Entrenamiento del modelo

Se utiliza el *accuracy* como una forma rápida de monitorear el progreso del entrenamiento en los datos de validación, sin embargo, se debe tener en cuenta que para problemas de traducción automática generalmente se utiliza el puntaje BLEU, así como otras métricas en lugar del *accuracy*.

Se recomienda dejar el modelo entrenando durante al menos 30 épocas.

In [17]:
epochs = 30

ckpt = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.h5',
                       monitor='val_loss',
                       verbose=1,
                       save_best_only=True)
transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

In [18]:
%%time
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[ckpt])

Epoch 1/30
1302/1302 [==============================] - ETA: 0s - loss: 1.7345 - accuracy: 0.3955
Epoch 1: val_loss improved from inf to 1.36802, saving model to weights.01-1.37.h5
1302/1302 [==============================] - 96s 70ms/step - loss: 1.7345 - accuracy: 0.3955 - val_loss: 1.3680 - val_accuracy: 0.4906
Epoch 2/30
1302/1302 [==============================] - ETA: 0s - loss: 1.3684 - accuracy: 0.5217
Epoch 2: val_loss improved from 1.36802 to 1.18675, saving model to weights.02-1.19.h5
1302/1302 [==============================] - 90s 69ms/step - loss: 1.3684 - accuracy: 0.5217 - val_loss: 1.1868 - val_accuracy: 0.5584
Epoch 3/30
1302/1302 [==============================] - ETA: 0s - loss: 1.2022 - accuracy: 0.5766
Epoch 3: val_loss improved from 1.18675 to 1.10110, saving model to weights.03-1.10.h5
1302/1302 [==============================] - 90s 69ms/step - loss: 1.2022 - accuracy: 0.5766 - val_loss: 1.1011 - val_accuracy: 0.5944
Epoch 4/30
1302/1302 [======================

## Decodificación de oraciones de prueba

Finalmente, para traducir oraciones nuevas, simplemente se ingresa al modelo la oración en inglés vectorizada, así como el token de destino `"[start]"`, luego se genera repetidamente el siguiente token hasta llegar al token `"[end]"`.

In [19]:
def decode_sequence(input_sentence):
    """Decodifica una secuencia de entrada en una frase en español.

    Argumentos:
        input_sentence: Una cadena, la frase en inglés a ser decodificada.

    Devuelve:
        Una cadena, la frase en español decodificada.
    """
    # Tokenizar la frase de entrada
    tokenized_input_sentence = eng_vectorization([input_sentence])
    # Inicializar la frase decodificada con el marcador de inicio
    decoded_sentence = "[start]"
    # Iterar hasta la longitud máxima de la frase decodificada
    for i in range(max_decoded_sentence_length):
        # Tokenizar la frase decodificada hasta el momento
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        # Obtener las predicciones del modelo para la siguiente palabra
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        # Tomar la palabra con mayor probabilidad
        sampled_token_index = np.argmax(predictions[0, i, :])
        # Convertir el índice de la palabra en la palabra en sí
        sampled_token = spa_index_lookup[sampled_token_index]
        # Añadir la palabra a la frase decodificada
        decoded_sentence += " " + sampled_token
        # Si se encontró el marcador de fin, terminar la decodificación
        if sampled_token == "[end]":
            break
    # Devuelve la oración decodificada
    return decoded_sentence

In [20]:
# Establecer el tamaño máximo de la frase decodificada
max_decoded_sentence_length = 20

# Crear un diccionario que permita buscar el índice del vocabulario en español a partir de un token
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))

Después de 30 épocas se obtienen los siguientes resultados:

In [23]:
# Seleccionamos algunos textos de prueba de inglés
test_eng_texts = [pair[0] for pair in test_pairs]

# Hacemos 10 iteraciones para elegir una oración de entrada al azar y traducirla
for _ in range(10):
    # Elegimos una oración de entrada al azar
    input_sentence = random.choice(test_eng_texts)
    # Traducimos la oración de entrada
    translated = decode_sequence(input_sentence)
    # Imprimimos la oración de entrada y la traducción
    print(input_sentence, translated, '\n')

The girl has a scarf around her neck. [start] la niña ha roto la [UNK] [end] 

There's no need to hurry. We have plenty of time. [start] no tienes suerte de casarse tenemos mucho tiempo [end] 

I'm eating now. [start] estoy comiendo ahora [end] 

I do not allow sleeping in class. [start] no [UNK] la del trabajo [end] 

I just want to be a normal person. [start] solo quiero ser una normal [end] 

Is this Tom's? [start] esto es de tom [end] 

You're a weird kid. [start] eres un niño extraño [end] 

You can't stay in here all day. [start] no puedes quedar aquí todo el día [end] 

Please remember that. [start] por favor [UNK] eso [end] 

I have no idea what Tom's problem is. [start] no tengo idea de cuál es el problema de tom [end] 

